# Prepare database

In [1]:
import os

dir_name = 'C:/Users/windo/Documents/ravdess-dataset/audio_speech_actors_01-24'

# Initialize a list to hold the file names
all_files = []

# Traverse all subfolders and files using os.walk()
for root, _, file_names in os.walk(dir_name):
    for file_name in file_names:
        file_path = os.path.join(root, file_name)
        if os.path.isfile(file_path):  # Check if it's a file
            all_files.append(file_path)

all_files = sorted(all_files)

# # Display the collected file names
# print('List of all files in the folders:')
# for file in all_files:
#     print(file)

# Emotions in the RAVDESS dataset
emotions = {
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

# desired csv columns:
# label/name(actor number, take, etc) emotion(class) feature1 feature2...
# or class at the end?
len(all_files)

1440

In [ ]:
import numpy as np

import sys
sys.path.append("..")
import plap

# Use plap for parameterization
fvector = plap.FeatureVector("asc","ass","asf","lat","tc","sc","hsc","hsd","hss","hsv")
# block_size_ms = block_size_samples/sr*1000 = 1024/48000*1000 ~= 21.34 ms
preprocessor = plap.Preprocessor(preemphasis_coeff=None, block_size=1024, window_type="hamming")

def extract_features(audio_path):
    plap.parameterize(audio_path=audio_path, fvector=fvector, preprocessor=preprocessor)
    return fvector.values

In [ ]:
# import csv

# # Define the CSV filename
# # fvector = plap.FeatureVector("asc","ass","asf","lat","tc","sc","hsc","hsd","hss","hsv")
# csv_filename = './python_csv/ravdess_bs2048hamming_ov50_nopreemph.csv'

# # Initialize the CSV file with headers
# headers = ['label', 'class'] + [f'{feature.upper()}' for feature in fvector.features]
# with open(csv_filename, mode='w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(headers)

In [4]:
import time
# Iterate over each audio file
initial_i = 0
# end_i = 150
i = initial_i
processed = 0
for filename in all_files[initial_i:]:
    if filename.endswith('.wav'):  # Ensure only audio files are processed
        
        start_time = time.time()
        # Extract label and class from the filename (customize this logic as needed)
        audio_filename = filename.split('\\')[-1]
        label = audio_filename.removesuffix('.wav')  # Use filename as label
        class_num = audio_filename.split('-')[2]
        class_name = emotions[class_num]
        
        print(f'Processing file {audio_filename}')

        # Extract 11 features from the audio file
        try:
            features = extract_features(filename)
            processed += 1
        except:
            print(f'Omitting file{i}')
            i += 1
            continue
        # features = np.zeros(11)

        # Prepare the row to be added to the CSV
        row = [label, class_name] + list(features)

        # Append the new row to the CSV file
        with open(csv_filename, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(row)
        print(f'Finished file{i} in {time.time() - start_time} s')
        i += 1

print("Feature extraction and CSV writing completed.")

Processing file 03-01-01-01-01-01-01.wav
Finished file0 in 6.074387073516846 s
Processing file 03-01-01-01-01-02-01.wav
Finished file1 in 0.23402953147888184 s
Processing file 03-01-01-01-02-01-01.wav
Finished file2 in 0.22067046165466309 s
Processing file 03-01-01-01-02-02-01.wav
Finished file3 in 0.21085858345031738 s
Processing file 03-01-02-01-01-01-01.wav
Finished file4 in 0.23865008354187012 s
Processing file 03-01-02-01-01-02-01.wav


C:\Users\windo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=2
  warnings.warn(


Omitting file5
Processing file 03-01-02-01-02-01-01.wav
Finished file6 in 0.7228457927703857 s
Processing file 03-01-02-01-02-02-01.wav
Finished file7 in 0.6124653816223145 s
Processing file 03-01-02-02-01-01-01.wav
Finished file8 in 0.5230128765106201 s
Processing file 03-01-02-02-01-02-01.wav
Finished file9 in 0.5768170356750488 s
Processing file 03-01-02-02-02-01-01.wav
Finished file10 in 0.6497247219085693 s
Processing file 03-01-02-02-02-02-01.wav
Finished file11 in 0.5867173671722412 s
Processing file 03-01-03-01-01-01-01.wav
Finished file12 in 0.41620564460754395 s
Processing file 03-01-03-01-01-02-01.wav
Finished file13 in 0.36870360374450684 s
Processing file 03-01-03-01-02-01-01.wav
Finished file14 in 0.5437510013580322 s
Processing file 03-01-03-01-02-02-01.wav
Finished file15 in 0.6291773319244385 s
Processing file 03-01-03-02-01-01-01.wav
Finished file16 in 0.596301794052124 s
Processing file 03-01-03-02-01-02-01.wav
Finished file17 in 0.5171146392822266 s
Processing file 

In [5]:
processed

1435